<a href="https://colab.research.google.com/github/JUANITOTELO/Proyecto_Agro_Proba/blob/main/entrega3estad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages('plyr')

In [ ]:
%%R

#Instalación y cargado de la librería necesaria para hacer el conteo 
#de frecuencias de variable categórica
library(plyr)
library(ggplot2)

## Cargado del csv principal para los análisis

In [ ]:
%%R
cultivos <- read.table("/gdrive/MyDrive/Universidad/proba/entrega2/cultivos_nacional_filtrado.csv", sep=",", header=TRUE)

head(cultivos)

  P_DEPTO     P_S6P46 P_S6P47B P_S6P59_UNIF P_S6P60 AREA_SEMBRADA
1       5   112201001     2012     3.277476      12     0.9541966
2       5   151001001     2013    16.022034      12     0.9541966
3       5   151001001     2012    14.948570      12     0.9541966
4       5   180201002     2007     3.852656      12     1.3533684
5       5   159201001     2013    18.642798      12     0.7151330
6       5 16100100101     1997     0.720000      12     0.7151330
  AREA_COSECHADA
1      0.9541966
2      0.9541966
3      0.9541966
4      1.3533684
5      0.7151330
6      0.7151330


----
Creación de un subdataframe para el primer análisis, a.k.a top 10 de los cultivos más afectados en el país

In [ ]:
%%R
#selección de las columnas que nos interesan para este análisis inicial
cultivos_sub_top10 <- subset(cultivos, select=c("P_S6P46", "P_S6P60"))
cultivos_sub_top10 = data.frame(cultivos_sub_top10)

head(cultivos_sub_top10)

      P_S6P46 P_S6P60
1   112201001      12
2   151001001      12
3   151001001      12
4   180201002      12
5   159201001      12
6 16100100101      12


In [ ]:
%%R
cultivos_afectados <- cultivos_sub_top10[cultivos_sub_top10$P_S6P60 != 12,]
head(cultivos_afectados)

       P_S6P46 P_S6P60
63   121903001       2
64   125402001       2
111  121903001      10
112  125402001      10
1615 125402001       2
1616 123401001       7


In [ ]:
%%R

freqs_cult_afect <- count(cultivos_afectados, "P_S6P46")
freqs_cult_afect <- freqs_cult_afect[order(-freqs_cult_afect$freq),]
freqs_cult_afect[0:10,]

      P_S6P46  freq
49  131301001 13876
3   112201002 12499
113 159201001 11001
2   112201001  9694
109 151001001  7666
48  131201001  5818
140 191999056  3320
135 180201002  3290
124 164001001  3256
4   113202001  3056


## carga de csv para obtener equivalencias de código de cultivo a nombre del cultivo

In [ ]:
%%R
cultivos_code2name <- read.table("/gdrive/MyDrive/Universidad/proba/entrega2/equivalencias_cultivos_code.csv", sep=",", header=TRUE)

head(cultivos_code2name)

    P_S6P46            nombre
1 196104035           Abarco 
2 191999067 Acacia Forrajera 
3 196104068           Acacio 
4 196105051   Aceite de maría
5 121904001           Acelga 
6 196104061          Achapo  


In [ ]:
%%R
#ahora se hace el merge entre las 2 tablas para poder conocer los nombres del top 10 
#de los cultivos más afectados

freqs_cult_afect_nombre = merge(x = cultivos_code2name, y=freqs_cult_afect, by = "P_S6P46", all=FALSE)
freqs_cult_afect_nombre = freqs_cult_afect_nombre[order(-freqs_cult_afect_nombre$freq),]
freqs_cult_afect_nombre[0:10,]

      P_S6P46         nombre  freq
49  131301001       Platano  13876
3   112201002 Maíz Amarillo  12499
113 159201001          Yuca  11001
2   112201001   Maíz Blanco   9694
109 151001001         Papas   7666
48  131201001        Banano   5818
140 191999056         Fríjol  3320
135 180201002  Caña panelera  3290
124 164001001    Cacao grano  3256
4   113202001   Arroz verde   3056


In [ ]:
%%R
pdf(file = "Cantidad de cultivos afectados (nivel nacional) (top 10)")
p <- ggplot(data = freqs_cult_afect_nombre[0:10,], aes(x=nombre, y=freq, fill=freq)) + geom_bar(stat="identity", color="black") +
     coord_flip() 
p <- p + labs(title="Cantidad de cultivos afectados (nivel nacional) (top 10)", x="Cultivo", y = "Cantidad de cultivos afectados")
p  + theme_minimal()

# Prueba de hipótesis para la diferencia de medias entre rendimientos de cultivos afectados y no afectados

In [ ]:
%%R
#selección de las columnas que nos interesan para este análisis inicial
cultivos_hipo <- subset(cultivos, select=c("P_S6P46", "P_S6P60", "P_S6P59_UNIF"))
cultivos_hipo = data.frame(cultivos_hipo)

head(cultivos_hipo)

      P_S6P46 P_S6P60 P_S6P59_UNIF
1   112201001      12     3.277476
2   151001001      12    16.022034
3   151001001      12    14.948570
4   180201002      12     3.852656
5   159201001      12    18.642798
6 16100100101      12     0.720000


In [ ]:
%%R

#selección de los cultivos afectados y los no afectados en grupos distintos
#y borrado de los NA en la columna P_S6P59_UNIF -> 'rendimientos', 
#también borra las filas que no tienen código
cultivos_hipo_afect <- cultivos_hipo[(cultivos_hipo$P_S6P60 != 12),]
cultivos_hipo_afect <- cultivos_hipo_afect[complete.cases(cultivos_hipo_afect[,3]),]
cultivos_hipo_afect <- cultivos_hipo_afect[complete.cases(cultivos_hipo_afect[,1]),]

cultivos_hipo_no_afect <- cultivos_hipo[(cultivos_hipo$P_S6P60 == 12),]
cultivos_hipo_no_afect <- cultivos_hipo_no_afect[complete.cases(cultivos_hipo_no_afect[,3]),]
cultivos_hipo_no_afect <- cultivos_hipo_no_afect[complete.cases(cultivos_hipo_no_afect[,1]),]

head(cultivos_hipo_no_afect)

      P_S6P46 P_S6P60 P_S6P59_UNIF
1   112201001      12     3.277476
2   151001001      12    16.022034
3   151001001      12    14.948570
4   180201002      12     3.852656
5   159201001      12    18.642798
6 16100100101      12     0.720000


In [ ]:
%%R
cultivos_hipo_no_afect = merge(x = cultivos_code2name, y=cultivos_hipo_no_afect, by = "P_S6P46", all=FALSE)
cultivos_hipo_afect = merge(x = cultivos_code2name, y=cultivos_hipo_afect, by = "P_S6P46", all=FALSE)

head(cultivos_hipo_afect)

    P_S6P46           nombre P_S6P60 P_S6P59_UNIF
1 111201001 Trigo en grano         7     3.432202
2 111201001 Trigo en grano        10     1.312838
3 111201001 Trigo en grano         5     3.277186
4 111201001 Trigo en grano        11     3.331678
5 111201001 Trigo en grano         7     3.317994
6 111201001 Trigo en grano         2     3.320472


In [ ]:
%%R
platano_triste = cultivos_hipo_afect[cultivos_hipo_afect$nombre == "Platano ",]
platano_feliz = cultivos_hipo_no_afect[cultivos_hipo_no_afect$nombre == "Platano ",]

head(platano_feliz)

         P_S6P46   nombre P_S6P60 P_S6P59_UNIF
578315 131301001 Platano       12     5.003673
578316 131301001 Platano       12     8.126980
578317 131301001 Platano       12     6.186887
578318 131301001 Platano       12     6.420560
578319 131301001 Platano       12     3.553809
578320 131301001 Platano       12     7.520362


In [ ]:
%%R
n1 = count(platano_feliz, "nombre")[,"freq"]
n2 = count(platano_triste, "nombre")[,"freq"]

head(n1)
head(n2)

[1] 13026


In [ ]:
%%R
mu_1 = mean(platano_feliz[,"P_S6P59_UNIF"])
mu_2 = mean(platano_triste[,"P_S6P59_UNIF"])

print(c(mu_1, mu_2))

[1] 5.458774 3.875260


In [ ]:
%%R
sd_1 = sd(platano_feliz[,"P_S6P59_UNIF"])
sd_2 = sd(platano_triste[,"P_S6P59_UNIF"])

print(c(sd_1, sd_2))

[1] 1.425433 1.697868


cálculo del estadístico de prueba (dado el tamaño de n se puede normalizar).

$\alpha = 0.05$

$h_o: \mu_1 - \mu_2 = 0$ 

$h_a: \mu_1 - \mu_2 \neq 0$ 

$RR: \{z: |z| > z_{\alpha/2} \}$

In [ ]:
%%R
std_err = sqrt( ((sd_1^2)/n1) + ((sd_2^2)/n2) )
std_err

[1] 0.01506192


In [ ]:
%%R
z_alfa_med = qnorm(1-0.025)
z = ( (mu_1 - mu_2) - (0))/std_err

print(c(z_alfa_med, z))

[1]   1.959964 105.133639


#### Intervalo de confianza

$\hat{\theta} \pm  z_{\alpha/2} \cdot \sigma_{\hat{\theta}}$

In [ ]:
%%R

intervalo_conf = c(z - z_alfa_med *std_err, z + z_alfa_med *std_err)
print(intervalo_conf)

[1] 105.1041 105.1632


Al analizar que este intervalo de confianza está por mucho dentro del rango positivo vemos que, $\mu_1 > \mu_2$, es decir que un cultivo no afectado tiene mejores rendimientos

In [ ]:
%%R
p_valor = pnorm(z, lower.tail =FALSE)*2
print(p_valor)

[1] 0


### Con lo anterior rechazamos rotundamente que la diferencia de medias es cero, luego que un cultivo se vea afectado por algún factor ambiental incide en su rendimiento

## cálculo del error $\beta$

In [ ]:
%%R

diff_alt = 6*std_err
print(diff_alt)

k = (z_alfa_med * std_err) + 0
print(k)

pnorm( (k- 6*std_err)/std_err )

[1] 0.09037151
[1] 0.02952082
[1] 2.67215e-05


Se puede concluir que con total certeza hay una diferencia considerable en las medias, al todos los parametros probabilisticos que hallamos decir que hay que rechazar rotundamente H0. Ademas por la naturaleza de los resultados se puede decir que M1 es mayor a M2 con total certeza, por lo cual podemos decir que definitivamente hay una reduccion en el rendimiento de los cultivos de Platano cuando son afectados por algun agente externo negativo. 

A partir de aqui haremos esta misma prueba de hipotesis pero para todas las especies de cultivos.

In [ ]:
%%R
cultivos_hipo_no_afect[1:10,]


     P_S6P46           nombre P_S6P60 P_S6P59_UNIF
1  111201001 Trigo en grano        12     2.063049
2  111201001 Trigo en grano        12     3.438222
3  111201001 Trigo en grano        12     2.360736
4  111201001 Trigo en grano        12     2.412808
5  111201001 Trigo en grano        12     3.411425
6  111201001 Trigo en grano        12     2.408018
7  111201001 Trigo en grano        12     3.411425
8  111201001 Trigo en grano        12     3.411425
9  111201001 Trigo en grano        12     3.102294
10 111201001 Trigo en grano        12     3.411425


In [ ]:
%%R
n_afect = aggregate(cultivos_hipo_afect$P_S6P59_UNIF, list(cultivos_hipo_afect$nombre), length)
n_no_afect = aggregate(cultivos_hipo_no_afect$P_S6P59_UNIF, list(cultivos_hipo_no_afect$nombre), length)

n_afect = rename(n_afect, replace = c("x" = "n_triste"))
n_no_afect = rename(n_no_afect, replace = c("x" = "n_feliz"))

n_afect[1:10,]

                   Group.1 n_triste
1                  Abarco         2
2                  Acacio         1
3                  Acelga        25
4                  Achiote       47
5                 Achiras        25
6  Agapanto (Flor de amor)        1
7                Aguacate       419
8                 Ahuyama       672
9       Ajíes y pimientos       362
10               Ajonjolí        45


In [ ]:
%%R
n_cults = merge(x = n_afect, y= n_no_afect, by = "Group.1", all=FALSE)
n_cults[1:10,]

                   Group.1 n_triste n_feliz
1                  Abarco         2      16
2                  Acacio         1     236
3                  Acelga        25     859
4                  Achiote       47     554
5                 Achiras        25   16164
6  Agapanto (Flor de amor)        1      55
7                Aguacate       419   37596
8                 Ahuyama       672   10627
9       Ajíes y pimientos       362    8008
10               Ajonjolí        45     223


In [ ]:
%%R
medias_afect = aggregate(cultivos_hipo_afect[,4], list(cultivos_hipo_afect$nombre), mean, na.rm=TRUE)
sd_afect = aggregate(cultivos_hipo_afect[,4], list(cultivos_hipo_afect$nombre), sd, na.rm=TRUE)

medias_afect = rename(medias_afect, replace = c("x" = "y_bar_triste"))
sd_afect = rename(sd_afect, replace = c("x" = "sd_triste"))

medias_afect[1:4,]
sd_afect[1:4,]

  Group.1 sd_triste
1 Abarco   0.000000
2 Acacio         NA
3 Acelga   4.192021
4 Achiote  0.000000


In [ ]:
%%R
medias_no_afect = aggregate(cultivos_hipo_no_afect[,4], list(cultivos_hipo_no_afect$nombre), mean, na.rm=TRUE)
sd_no_afect = aggregate(cultivos_hipo_no_afect[,4], list(cultivos_hipo_no_afect$nombre), sd, na.rm=TRUE)

medias_no_afect = rename(medias_no_afect, replace = c("x" = "y_bar_feliz"))
sd_no_afect = rename(sd_no_afect, replace = c("x" = "sd_feliz"))


medias_no_afect[1:4,]
sd_no_afect[1:4,]

          Group.1  sd_feliz
1         Abarco  0.9363562
2         Acacio  2.0033063
3 Aceite de maría 1.1602724
4         Acelga  4.6103268


In [ ]:
%%R
medias_rendimiento = merge(x = medias_afect, y= medias_no_afect, by = "Group.1", all=FALSE)
medias_rendimiento[1:4,]

  Group.1 y_bar_triste y_bar_feliz
1 Abarco        8.4000    8.024050
2 Acacio        6.6000    6.382595
3 Acelga       10.6024    9.975504
4 Achiote       0.5000    0.500000


In [ ]:
%%R
sd_rendimiento = merge(x = sd_afect, y= sd_no_afect, by = "Group.1", all=FALSE)
sd_rendimiento = sd_rendimiento[complete.cases(sd_rendimiento[,3]),]
sd_rendimiento[1:4,]

  Group.1 sd_triste  sd_feliz
1 Abarco   0.000000 0.9363562
2 Acacio         NA 2.0033063
3 Acelga   4.192021 4.6103268
4 Achiote  0.000000 0.0000000


In [ ]:
%%R
sd_med_rend = merge(x = sd_rendimiento, y= medias_rendimiento, by = "Group.1", all=FALSE)
sd_med_rend[1:10,]

                   Group.1  sd_triste   sd_feliz y_bar_triste y_bar_feliz
1                  Abarco  0.00000000 0.93635621    8.4000000   8.0240504
2                  Acacio          NA 2.00330632    6.6000000   6.3825951
3                  Acelga  4.19202071 4.61032678   10.6023977   9.9755042
4                  Achiote 0.00000000 0.00000000    0.5000000   0.5000000
5                 Achiras  2.24588448 0.74005406    3.6560891   1.4512051
6  Agapanto (Flor de amor)         NA 9.82827670    7.1428571  15.0903706
7                Aguacate  1.73325644 1.64303531    3.5714134   6.0267323
8                 Ahuyama  2.24273426 3.13125842    4.2762258   5.8195051
9       Ajíes y pimientos  2.53698915 3.28116518    3.4030219   5.3608856
10               Ajonjolí  0.04648659 0.02116515    0.3102557   0.2920708


In [ ]:
%%R
sd_med_n =  merge(x = n_cults, y= sd_med_rend, by = "Group.1", all=FALSE)
sd_med_n[1:4,]

  Group.1 n_triste n_feliz sd_triste  sd_feliz y_bar_triste y_bar_feliz
1 Abarco         2      16  0.000000 0.9363562       8.4000    8.024050
2 Acacio         1     236        NA 2.0033063       6.6000    6.382595
3 Acelga        25     859  4.192021 4.6103268      10.6024    9.975504
4 Achiote       47     554  0.000000 0.0000000       0.5000    0.500000


In [ ]:
%%R
medias_diff = sd_med_n["y_bar_feliz"] - sd_med_n["y_bar_triste"]
sd_med_n["medias_diff"] = medias_diff
sd_med_n[1:4,]

   Group.1 n_triste n_feliz sd_triste  sd_feliz y_bar_triste y_bar_feliz
1  Abarco         2      16  0.000000 0.9363562     8.400000    8.024050
3  Acelga        25     859  4.192021 4.6103268    10.602398    9.975504
4  Achiote       47     554  0.000000 0.0000000     0.500000    0.500000
5 Achiras        25   16164  2.245884 0.7400541     3.656089    1.451205
  medias_diff
1  -0.3759496
3  -0.6268936
4   0.0000000
5  -2.2048840


In [ ]:
%%R
#borrado de los na que impiden calcular
sd_med_n = sd_med_n[complete.cases(sd_med_n[,4]),]

#selección de los cultivos que son normalizables 
#dado que tienen n > 40
sd_med_n_norm = sd_med_n[sd_med_n$n_triste > 60,]
sd_med_n_norm = sd_med_n_norm[sd_med_n_norm$n_feliz > 60,]

sd_med_n_norm[1:4,]

              Group.1 n_triste n_feliz sd_triste  sd_feliz y_bar_triste
7           Aguacate       419   37596 1.7332564 1.6430353     3.571413
8            Ahuyama       672   10627 2.2427343 3.1312584     4.276226
9  Ajíes y pimientos       362    8008 2.5369891 3.2811652     3.403022
14           Algodón       654    7707 0.4001618 0.3861851     2.452522
   y_bar_feliz medias_diff
7     6.026732  2.45531897
8     5.819505  1.54327925
9     5.360886  1.95786377
14    2.373022 -0.07949948


In [ ]:
%%R

sd_med_n_norm["std_err"] = sqrt((sd_med_n_norm$sd_feliz^2)/sd_med_n_norm$n_feliz + 
                                (sd_med_n_norm$sd_triste^2)/sd_med_n_norm$n_triste)

In [ ]:
%%R

sd_med_n_norm["z"] = sd_med_n_norm$medias_diff/sd_med_n_norm$std_err

In [ ]:
%%R 
#para un alfa de 0.05
alfa_med = 0.05
z_alfa = qnorm(alfa_med)

sd_med_n_norm["aceptar_ha"] = sd_med_n_norm$z < z_alfa
sd_med_n_norm["p_valor"] = pnorm((sd_med_n_norm$z), lower.tail = TRUE)


sd_med_n_norm["aceptar_ha(p_val)"] = sd_med_n_norm$p_valor < alfa_med
#aceptar_p

In [ ]:
%%R 
z_final = sd_med_n_norm[,c("Group.1","aceptar_ha","aceptar_ha(p_val)", "z")]
z_final

In [ ]:
%%R

#calcular el porcentaje de todos los cultivos para los cuales se cumple
#la hipótesis alternativa

total = length(sd_med_n_norm$Group.1)

sis = sd_med_n_norm[sd_med_n_norm$aceptar_ha == TRUE,]
sis = length(sis$Group.1)

ratio_ha_aceptada = sis*100/total
ratio_ha_aceptada 

[1] 87.5


In [ ]:
%%R
fresa = sd_med_n_norm[sd_med_n_norm$Group.1 == "Fresa ",]
fresa

   Group.1 n_triste n_feliz sd_triste sd_feliz y_bar_triste y_bar_feliz
86  Fresa        73    3879  9.732229 2.268184     4.300443    6.376523
   medias_diff  std_err        z aceptar_ha    p_valor aceptar_ha(p_val)
86     2.07608 1.139653 1.821677      FALSE 0.03425202             FALSE


A partir de aqui haremos la distribucion t con aquellos cultivos con n menos a 60. Los de mayor a 60 ya les hicimos la prueba z.

$t = \frac{y_1-y_2-0}{sp \sqrt{\frac{1}{n_1}+\frac{1}{n_2}}}$

$sp = \sqrt{\frac{(n_1-1)s_1^2+(n_2-1)s_2^2}{n_1+n_2-2}}$

In [ ]:
%%R

sd_med_n_tstudent = sd_med_n[sd_med_n$n_triste < 60,]
sd_med_n_tstudent = sd_med_n_tstudent[sd_med_n_tstudent$n_feliz  < 60,]

sd_med_n_tstudent[1:4,]

In [ ]:
%%R

sd_med_n_tstudent["df"] = sd_med_n_tstudent$n_feliz + sd_med_n_tstudent$n_triste - 2

sd_med_n_tstudent["sp"] = sqrt(((sd_med_n_tstudent$n_feliz-1)*(sd_med_n_tstudent$sd_feliz)^2 + (sd_med_n_tstudent$n_triste-1)*(sd_med_n_tstudent$sd_triste)^2)/
                               (sd_med_n_tstudent$df))



In [ ]:
%%R

sd_med_n_tstudent["tstudent"] = (sd_med_n_tstudent$medias_diff)/(sd_med_n_tstudent$sp*(sqrt((1/sd_med_n_tstudent$n_feliz) + (1/sd_med_n_tstudent$n_triste))))

In [ ]:
%%R 
#para un alfa de 0.05
alfa_med_tstudent = 0.05
tstudent_alfa = qt(alfa_med_tstudent, df = sd_med_n_tstudent$df)

sd_med_n_tstudent["aceptar_ha"] = sd_med_n_tstudent$tstudent < tstudent_alfa
sd_med_n_tstudent["p_valor"] = pt((sd_med_n_tstudent$tstudent), df = sd_med_n_tstudent$df, lower.tail = TRUE)


sd_med_n_tstudent["aceptar_ha(p_val)"] = sd_med_n_tstudent$p_valor < alfa_med
#aceptar_p

In [ ]:
%%R 
tstudent_final = sd_med_n_tstudent[,c("Group.1","aceptar_ha","aceptar_ha(p_val)", "tstudent")]
tstudent_final

In [ ]:
%%R

write.csv(x = z_final, file = "/gdrive/MyDrive/Tabla_PdHMedias_Z.csv", row.names = TRUE)
write.csv(x = tstudent_final, file = "/gdrive/MyDrive/Tabla_PdHMedias_Tstudent.csv", row.names = TRUE)

Ahora a partir de aqui graficamos las proporciones de la prueba de hipotesis 

In [ ]:
%%R 

tstudent_final_porcentaje = tstudent_final[,c("Group.1","aceptar_ha")]
z_final_porcentaje = z_final[,c("Group.1","aceptar_ha")]

tabla_porcentaje = rbind(tstudent_final_porcentaje, z_final_porcentaje)
tabla_porcentaje

In [ ]:
%%R

porcentaje_h0 = round((51 * 100 / 74), 0)
porcentaje_ha = round((23 * 100 / 74), 0)
pdf("Piecharthipotesis.pdf")
pie(table(tabla_porcentaje$aceptar_ha), labels = paste0(c(porcentaje_h0, porcentaje_ha), "%"), main = ("Porcentajes sobre la prueba de hipotesis"))

legend("topleft", legend = c("Aceptamos H0", "Aceptamos Ha"),
       fill =  c("white", "lightblue"))
